In [ ]:
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
from scipy import signal as sp

from turtle_quant_1.data_processing.processor import DataProcessor
from turtle_quant_1.strategies.helpers.helpers import convert_to_daily_data

In [ ]:
dates = {"start": datetime(2024, 1, 1), "end": datetime(2025, 7, 31)}

data_processor = DataProcessor()
data = data_processor.load_data(
    symbol="MSFT",
    start_date=dates["start"],
    end_date=dates["end"],
    impute_data=True,
)

In [ ]:
# This step is not necessary, but just so that it is easier to see what data the strategies are processing
agg_data = convert_to_daily_data(data)

In [ ]:
# Adjust this to configure what price counts as being within a support-resistance zone
sup_res_zone_threshold = 0.005

In [ ]:
peak_distance = 10
peak_prominence_pct = 0.01

high_prices = agg_data["High"].values
low_prices = agg_data["Low"].values
close_prices = agg_data["Close"].values

# Calculate dynamic prominence based on price range
price_range = np.max(high_prices) - np.min(low_prices)
prominence_threshold = price_range * peak_prominence_pct

peaks, _ = sp.find_peaks(
    close_prices,
    distance=peak_distance,
    prominence=prominence_threshold,
)

troughs, _ = sp.find_peaks(
    -close_prices,
    distance=peak_distance,
    prominence=prominence_threshold,
)

In [ ]:
peaks

In [ ]:
agg_data.iloc[peaks].head()

In [ ]:
agg_data["ma"] = agg_data["Close"].rolling(window=10).mean()

In [ ]:
import plotly.graph_objects as go
from IPython.display import display
from plotly.subplots import make_subplots

# Create subplots
fig = make_subplots(
    rows=1,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.1,
    row_heights=[1.0],
    subplot_titles=("Close Price",),
)

# # Price plot with original data
# fig.add_trace(
#     go.Scatter(
#         x=data["datetime"],
#         y=data["Close"],
#         mode="lines",
#         name="Close Price",
#         line=dict(color="blue"),
#     ),
#     row=1,
#     col=1,
# )

# Price plot with aggregated data
fig.add_trace(
    go.Scatter(
        x=agg_data["datetime"],
        y=agg_data["Close"],
        mode="lines",
        name="Aggregated Close",
        line=dict(color="blue"),
    ),
    row=1,
    col=1,
)

# Peak markers
fig.add_trace(
    go.Scatter(
        x=agg_data.iloc[peaks]["datetime"],
        y=agg_data.iloc[peaks]["Close"],
        mode="markers",
        name="SciPy Peaks",
        marker=dict(color="green", size=8),
        showlegend=False,
    ),
    row=1,
    col=1,
)

# Trough markers
fig.add_trace(
    go.Scatter(
        x=agg_data.iloc[troughs]["datetime"],
        y=agg_data.iloc[troughs]["Close"],
        mode="markers",
        name="SciPy Troughs",
        marker=dict(color="red", size=8),
        showlegend=False,
    ),
    row=1,
    col=1,
)

# 20-day Moving Average
fig.add_trace(
    go.Scatter(
        x=agg_data["datetime"],
        y=agg_data["ma"],
        mode="lines",
        name="20-Day MA",
        line=dict(color="orange"),
    ),
    row=1,
    col=1,
)

# Layout
fig.update_layout(
    height=600,
    width=1000,
    title_text="MSFT",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
)

fig.update_yaxes(title_text="Price", row=1, col=1)

display(fig)